# Grover's search algorithm

In [1]:
# Number of qubits in index register
indexLength = 8
# Number of qubits in value register
valueLength = 8
# Carry flag qubit index
carryIndex = indexLength + valueLength
# Value we're searching for
TARGET_VALUE = 100

In [2]:
# (We don't "know" this, for purpose of the problem.)
TARGET_KEY = 230

In [3]:
from pyqrack import QrackSimulator
sim = QrackSimulator(indexLength + valueLength + 1)

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_Gen9_HD_Graphics_NEO.ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir


In [4]:
toLoad = [1] * (1 << indexLength)
toLoad[TARGET_KEY] = TARGET_VALUE

In [5]:
indexQubits = [i for i in range(indexLength)]
valueQubits = [(v + indexLength) for v in range(valueLength)]
indexControls = [i for i in range(1, indexLength)]
valueControls = [(v + indexLength) for v in range(1, valueLength)]

In [6]:
for i in range(indexLength):
    sim.h(i)

In [7]:
sim.lda(indexQubits, valueQubits, toLoad)

In [8]:
import math
optIter = math.floor(math.pi / (4 * math.asin(1 / math.sqrt(1 << indexLength))))

In [9]:
for i in range(optIter):
    sim.sub(TARGET_VALUE, valueQubits)
    sim.macmtrx(valueControls, [-1, 0, 0, 1], indexLength)
    sim.add(TARGET_VALUE, valueQubits)

    sim.x(carryIndex)
    sim.sbc(carryIndex, indexQubits, valueQubits, toLoad)
    sim.x(carryIndex)
    for i in indexQubits:
        sim.h(i)
    sim.macmtrx(phaseFlipControls, [-1, 0, 0, 1], 0)
    for i in indexQubits:
        sim.h(i)
    # We could "reverse sign of global phase," here, but that cannot be measured.
    sim.adc(carryIndex, indexQubits, valueQubits, toLoad)

In [10]:
print(sim.measure_shots(valueQubits, 10))
print(sim.measure_shots(indexQubits, 10))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 2, 2, 2]


**The above result is wrong. This notebook and supporting packages are still in development.**